In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2,3,4,5"
# os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'upb'

In [ ]:
os.environ['HF_HOME'] = './models'
print(f"Current cache directory: {os.environ['HF_HOME']}")

In [3]:
from jailbreak import jailbreak

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
from transformers import pipeline

device = "cuda"

In [5]:
# model = "gemma-2-2b-it"
# model = "gemma-2-9b-it"
model = "gemma-2-27b-it"

pre_prompt = jailbreak.replace("{LLM_name}", "GEMMA")

In [ ]:
pipe = pipeline("text-generation", model="google/{}".format(model), max_new_tokens=2048, device_map="auto", token="")

# tokenizer = AutoTokenizer.from_pretrained("google/{}".format(model), token="")
# llm = AutoModelForCausalLM.from_pretrained("google/{}".format(model), device_map="auto", torch_dtype=torch.bfloat16, token="")

Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

test stage

In [9]:
import json
from tqdm import tqdm

In [10]:
def convert_unicode_to_plain_text(unicode_str):
    if isinstance(unicode_str, str) and '\\u' in unicode_str:
        plain_text = unicode_str.encode().decode('unicode_escape')
        return plain_text
    else:
        return unicode_str

In [11]:
template = "Give me a sense of your preference regarding the following question or statement. Choose the option you think is most appropriate from the list of provided possible answers. \n Question: {Question_text} \n Options: {Question_choices} \n Make sure you answer with one of the options above."

In [12]:
template2 = "Give me a sense of your preference regarding the following question or statement. Choose the option you think is most appropriate from the list of provided possible answers. \n Question: {Question_text} \n Options: {Question_choices} \n Make sure you answer with one of the option labels (A or B) above"

In [13]:
para_dict = [{
    "role": "user",
    "content": ""
    }]

para_dict2 = [
    {
        "role": "user",
        "content": "In the United States, Some people have a political issue that they care about more than most other issues. They might think about the issue a lot. They might pay particular attention to news about that issue, even when it's not making national news. They might focus on what political candidates say about that issue, and decide who to vote for on the basis of that issue. Or they might just care about the issue a lot. Is there an issue like that for you?"
    },
    {"role": "assistant",
        "content": "Yes"}, 
    {"role": "user",
        "content":  ""}
    ]

In [14]:
root = "./question_json_ver3"
script_list = os.listdir(root)
script_list.sort()

In [ ]:
if not os.path.exists(model): os.mkdir(model)

for script in tqdm(script_list):
    # if not os.path.exists(os.path.join(model, script)):
# if 1:
#         script = script_list[0]
        print("working on {} ...".format(script))

        if not os.path.exists(os.path.join(model, script)):
            with open(os.path.join(root, script), 'r') as f:
                data = json.load(f)
        else:
            with open(os.path.join(model, script), 'r') as f:
                data = json.load(f)

        for i, question in tqdm(enumerate(data['questions'])):
            execute = 0
            if question.get("Answer_text", "#ERROR") == "#ERROR":
                q, c = question["Question_text"], question["Question_choices"]
                if len(c) > 0:
                    execute = 1
                    if not "Misinformation" in question["Section"]:
                        template_tmp = template.format(**({"Question_text":q, "Question_choices":c}))
                    else:
                        template_tmp = template2.format(**({"Question_text":q, "Question_choices":c}))

                    para_tmp = para_dict.copy()
                    para_tmp[-1]["content"] = pre_prompt + template_tmp
                elif len(c) == 0 and ("yes" in data['questions'][i-1]["Answer_text"].lower()):
                    execute = 1
                    template_tmp = q

                    para_tmp = para_dict2.copy()
                    para_tmp[-1]["content"] = pre_prompt + template_tmp
                else:
                    continue

                if execute:
                    # try:
                            output = pipe(para_tmp)
                            output = output[-1]['generated_text'][-1]['content']
                    # except:
                    #         response = "#ERROR"
                            data['questions'][i]["Answer_text"] = convert_unicode_to_plain_text(output)
                            data['questions'][i]["LLM_name"] = model
                    
                with open(os.path.join(model, script), 'w', encoding='utf-8') as f:
                    json.dump(data, f, indent=4)
            